In [1]:
import torch
import numpy as np

from torch import nn
from torch.nn import functional
from torch.autograd import Variable

from sklearn import datasets
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

True

In [4]:
iris = datasets.load_iris()

In [5]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                    iris.target,
                                                    test_size = 0.2,
                                                    random_state = 101010)

In [7]:
class MLP(nn.Module):

  def __init__(self):

    super(MLP, self).__init__()

    self.W1 = nn.Linear(4, 2)
    self.W2 = nn.Linear(2, 3)

    self.softmax = nn.Softmax(dim = 1)

  def forward(self, X):

    X = functional.relu(self.W1(X))
    X = self.W2(X)
    X = self.softmax(X)

    return X


In [15]:
X_train = Variable(torch.Tensor(X_train).float().cuda())
X_test = Variable(torch.Tensor(X_test).float().cuda())
y_train = Variable(torch.Tensor(y_train).long().cuda())
y_test = Variable(torch.Tensor(y_test).long().cuda())

In [16]:
X_train.shape, X_test.shape

(torch.Size([120, 4]), torch.Size([30, 4]))

In [20]:
criterion = nn.CrossEntropyLoss()

In [21]:
mlp = MLP()
mlp.cuda()
optimizer = torch.optim.SGD(mlp.parameters(), lr = 0.01)

In [22]:
for iteration in range(50000):

  optimizer.zero_grad()
  output = mlp(X_train) # forward pass
  error = criterion(output, y_train)
  error.backward()
  optimizer.step()

  if iteration % 500 == 0:
    print('Iteration %d: error = %2.2f' % (iteration, error.data))

Iteration 0: error = 1.11
Iteration 500: error = 1.09
Iteration 1000: error = 0.99
Iteration 1500: error = 0.89
Iteration 2000: error = 0.87
Iteration 2500: error = 0.85
Iteration 3000: error = 0.83
Iteration 3500: error = 0.80
Iteration 4000: error = 0.78
Iteration 4500: error = 0.76
Iteration 5000: error = 0.74
Iteration 5500: error = 0.73
Iteration 6000: error = 0.71
Iteration 6500: error = 0.70
Iteration 7000: error = 0.69
Iteration 7500: error = 0.68
Iteration 8000: error = 0.68
Iteration 8500: error = 0.67
Iteration 9000: error = 0.66
Iteration 9500: error = 0.66
Iteration 10000: error = 0.65
Iteration 10500: error = 0.65
Iteration 11000: error = 0.65
Iteration 11500: error = 0.64
Iteration 12000: error = 0.64
Iteration 12500: error = 0.64
Iteration 13000: error = 0.63
Iteration 13500: error = 0.63
Iteration 14000: error = 0.63
Iteration 14500: error = 0.63
Iteration 15000: error = 0.63
Iteration 15500: error = 0.62
Iteration 16000: error = 0.62
Iteration 16500: error = 0.62
Iter

In [21]:
predicted_p = mlp(X_test)

In [24]:
p_max, y_pred = torch.max(predicted_p, 1)

In [28]:
y_pred

tensor([0, 0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 2, 0, 2, 0, 1, 0, 1, 2, 0, 1, 0, 1, 1,
        0, 0, 2, 2, 0, 2])

In [29]:
y_test

tensor([0, 0, 1, 0, 1, 1, 2, 2, 1, 1, 0, 2, 0, 2, 0, 1, 0, 1, 2, 0, 1, 0, 1, 1,
        0, 0, 2, 2, 0, 2])

In [30]:
confusion_matrix(y_test, y_pred)

array([[12,  0,  0],
       [ 0, 10,  0],
       [ 0,  0,  8]])

In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         8

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

